# Shor's factoring algorithm
This notebook exemplifies using [Quaspy](https://github.com/ekera/quaspy) to simulate Shor's general factoring algorithm [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700), modified as in [[E24]](https://doi.org/10.1145/3655026), and with the classical post-processing from [[E21b]](https://doi.org/10.1007/s11128-021-03069-1) and [[E24]](https://doi.org/10.1145/3655026).
It furthermore exemplifies using [Quaspy](https://github.com/ekera/quaspy) to simulate Seifert's variation [[Seifert01]](https://doi.org/10.1007/3-540-45353-9_24) of Shor's general factoring algorithm [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700), as described in [[E24t]](https://diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sect. 5.4) and [[E21]](https://doi.org/10.1515/jmc-2020-0006) (see App. A), with the classical post-processing from [[E21b]](https://doi.org/10.1007/s11128-021-03069-1), [[E24t]](https://diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sect. 5.4) and [[E21]](https://doi.org/10.1515/jmc-2020-0006) (see App. A) with supporting functions from [[E24]](https://doi.org/10.1145/3655026).

To start off, let us pick $n$ prime factors $p_i$ uniformly at random from the set of all $l$-bit prime factors, and $n$ exponents $e_i$ uniformly at random from $[1, e_{\max}] \cap \mathbb Z$, for $i \in [1, n] \cap \mathbb Z$, and take the product to form

$$N = \prod_{i=1}^n p_i^{e_i}.$$

To this end, we use the [<code>sample_l_bit_prime(l)</code>](../docs/math/primes/sample_l_bit_prime.md) and [<code>sample_integer(B)</code>](../docs/math/random/sample_integer.md) convenience functions provided by [Quaspy](https://github.com/ekera/quaspy).

In [1]:
!pip3 install -q --pre quaspy # Make sure that quaspy is installed.

In [2]:
from quaspy.math.primes import sample_l_bit_prime;
from quaspy.math.random import sample_integer;

from math import prod;

n = 10;
l = 1024;
e_max = 1;

factors = [];
primes = set();

while len(factors) < n:
  pi = sample_l_bit_prime(l);
  ei = sample_integer(e_max) + 1;

  if pi not in primes:
    primes.add(pi);
    factors.append([pi, ei]);

    print("Sampled p" + str(len(factors)) + " =", pi);

N = prod([pi ** ei for [pi, ei] in factors]);

print("\nComputed N =", N);

Sampled p1 = 157488917571705273509270310699508115691021449713982452084799674984076039577240666501260879197614082786627782313409064168693509545259317952900275098525256169978111944724257954903072358114089418323035111312077216514114295376514016129855517384969510491651605560206062459799015246918859314491944796022439311340099
Sampled p2 = 144958118968180212979415952332115357509995473424719933515342657913595128714245051098252347304570800859605505878138915463456918015964612094616850461010180401315663443458875107448964508634639867078442419041275280020006408320587757754707997348256076112158632407094932447922467042972730319918330675936103392741631
Sampled p3 = 105355216578965480113770819611745486300759363887585345573839215885601857663829844334065732682081875820802081223303914180669702782878687934328526668669094406673839473243502452621074916172144816682034121125225135016692101709855785248465053256221694974673307653712297495146390803485095744571092844347402290168923
Sampled p4 = 156913881488896793

## 1. Simulating order finding in $\mathbb Z_N^*$ exactly to sample $r$
[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_r_given_N(N, factors)</code>](../docs/factoring/sampling/sample_r_given_N.md) for exactly sampling an element $g$ uniformly at random from $\mathbb Z_N^*$ and returning its order without explicitly computing and returning $g$.
This function requires the factorization of $N$ to be known, as is the case here given that we select the factors of $N$.
For further details, see [[E24t]](https://diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sect. 5.2.3).

Below, we use the [<code>sample_r_given_N(N, factors)</code>](../docs/factoring/sampling/sample_r_given_N.md) function to simulate the initial classical pre-processing step of Shor's factoring algorithm where an element $g$ is selected uniformly at random from $\mathbb Z_N^*$.

In [3]:
from quaspy.factoring.sampling import sample_r_given_N;

r = sample_r_given_N(N, factors);

print("Sampled r =", r);

Sampled r = 3301343665518943159177062115067455094541376813626851486098602505479861624650115334195357116008122247662387145826047077045080790363893953966360515258467802528844701185387341503904827101496248494078597372767850421960489342442980160411196214842411446618564205855225129181674514424440761792653821821524219842202951897897778034214674015631884789689811535852616297191255352121827817675390554803640647559169439239876992380034308062090066086012175933969381527077163235034497794090751325096178751751603706798769883530974488902215932841425969965890835531558185044005423001549975491828056358048323110968187744208947362065676469298130547369785775298511466621180020608627965405425949452047832823558510008342847453929720663414825726551947489019531428857311101268215378271614209673317440776960614231885596553691949631926784261433956040865313858579404374948188110610991461261251747112801979066802379613462421411103732554142761855460808236804415725275528692111473478996186019394525178500813860452383256665

### 1.1. Solving $r$ for the complete factorization of $N$
We may then use the [<code>solve_r_for_factors(r, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_r_for_factors.md) function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $r$ for the factors of $N$ by using the post-processing from [[E21b]](https://doi.org/10.1007/s11128-021-03069-1).

In [4]:
from quaspy.factoring.general.postprocessing.ekera import solve_r_for_factors;

result = solve_r_for_factors(r, N);

print("Solving for the prime factors yielded:", result);

if result == primes:
  print("\n[ OK ] All prime factors of N were successfully recovered.");
else:
  print("\n[FAIL] The integer N was not completely factored.");

Solving for the prime factors yielded: {173612187966310048856533613919926723087410601579705334688318873344446043800050086085463822075235885182668014813003192310461613598155714480863906069098182045983142941158290019457107943528945346334700881758785143408420080471582745157047345541455016294240235708893368490542323437761887961141228197542063401693713, 98405689366658699793740774671388493146399314400366756950640922347844084251689300655531810820393726436218440023079778470944568991608249661166053747873038766882466781616445228750143893794510255140053414809393058202764441719028847452403794372876513937191571337099442917081948880823950633862187955099335442580059, 156913881488896793539425851866011077868395796772846710602560253453402314254064134697704979374584861997143601521530519128992332730503077660140706640878131189643875152158677158085019231148269289683451477255532537497954528933775907770617416686658637312536225636522693738347644676419707164328225753462105681945077, 1449581189681802129794159523

## 2. Simulating Shor's order-finding algorithm to sample a frequency $j$
[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_j_given_r(r, m, l, ..)</code>](../docs/orderfinding/general/sampling/sample_j_given_r.md) for simulating Shor's order-finding algorithm (and Seifert's variation thereof) exactly (up to arbitrary precision) for a given order $r$.
For further details, see [[E24t]](https://diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sects. 5.3.5 and 5.4.3).

Below, we use said function to simulate running the algorithm for $r$ with a control register of $m + \ell$ bits, where $m$ is an upper bound on the bit length of $r$, and $\ell$ is a positive integer.

More specifically, for $g$ a generator of order $r$, said function simulates inducing the state

$$\frac{1}{2^{m+\ell}}
\sum_{a, \, j \, = \, 0}^{2^{m+\ell} - 1}
\mathrm{exp}
\left(
  \frac{2 \pi \mathrm{i}}{2^{m + \ell}} aj
\right)
|\, j, g^a \,\rangle$$

and reading out the first control register.
This yields a frequency $j$ sampled from the probability distribution induced by the quantum part of the order-finding algorithm:

In [5]:
from quaspy.orderfinding.general.sampling import sample_j_given_r;

import sys;
sys.set_int_max_str_digits(10000);

m = N.bit_length() - 1;
l = m; # Ensures that r^2 < 2^(m + l) as r < 2^m;

j = sample_j_given_r(r, m, l, timeout = 30);

print("Sampled j =", j);

Sampled j = 7936554223008155815353011199314034125739153253171392353732469478881713776049554853095860934407820227984067889187353755265106996631719299838544464163420202679351192909279010811309495360444688092312987119505484163877829259013714923588341783946280946511897631302583736879009882840903772716829190732627757827950309293606992182715295584239091340217445203059515651040614639992798921978047154165613661666104796784197492659395704266869400804925740834154424575706717062862999311508393876639621551850989410872791771932312701162570526341026601792584977915248881201520475579428637610752398810178494145197979870217289847430367422608898783364588147540071947957205025561136716637871437771571840368960997699879170446779822439430461119788655566670067353359048580830374097545746208518368803858767664208527030022038891453309162046093766298355901390376974130800997468414565247192192022303972574604124329605240846582806830047076832693214871931535296912024797187081496379074982496370475896784319083780122132414

### 2.1. Solving the frequency $j$ and $g$ for the complete factorization of $N$
We now proceed to solve $j$ for the factors of $N$.

To this end, we first use the [SimulatedCyclicGroupElement](../docs/math/groups/SimulatedCyclicGroupElement.md) class provided by [Quaspy](https://github.com/ekera/quaspy) to define a group element $g$ of order $r$:

In [6]:
from quaspy.math.groups import SimulatedCyclicGroupElement;

g = SimulatedCyclicGroupElement(r);

We may then e.g. use the [<code>solve_j_for_factors(j, m, l, g, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_j_for_factors.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to first solve $j$ and $g$ for a positive integer multiple of $r$ using the lattice-based post-processing from [[E24]](https://doi.org/10.1145/3655026), and to then solve $r$ for the prime factors of $N$ using the post-processing from [[E21b]](https://doi.org/10.1007/s11128-021-03069-1):

In [7]:
from quaspy.factoring.general.postprocessing.ekera import solve_j_for_factors;

result = solve_j_for_factors(j, m, l, g, N);

print("Solving for the prime factors yielded:", result);

if result == primes:
  print("\n[ OK ] All prime factors of N were successfully recovered.");
else:
  print("\n[FAIL] Failed to completely factor the integer N.");

Solving for the prime factors yielded: {173612187966310048856533613919926723087410601579705334688318873344446043800050086085463822075235885182668014813003192310461613598155714480863906069098182045983142941158290019457107943528945346334700881758785143408420080471582745157047345541455016294240235708893368490542323437761887961141228197542063401693713, 98405689366658699793740774671388493146399314400366756950640922347844084251689300655531810820393726436218440023079778470944568991608249661166053747873038766882466781616445228750143893794510255140053414809393058202764441719028847452403794372876513937191571337099442917081948880823950633862187955099335442580059, 156913881488896793539425851866011077868395796772846710602560253453402314254064134697704979374584861997143601521530519128992332730503077660140706640878131189643875152158677158085019231148269289683451477255532537497954528933775907770617416686658637312536225636522693738347644676419707164328225753462105681945077, 1449581189681802129794159523

We may of course also solve in two steps, by first calling [<code>solve_j_for_r(j, m, l, g, ..)</code>](../docs/orderfinding/general/postprocessing/ekera/solve_j_for_r.md) to solve $j$ and $g$ for a positive integer multiple of $r$ using the lattice-based post-processing from [[E24]](https://doi.org/10.1145/3655026), and by then calling [<code>solve_r_for_factors(r, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_r_for_factors.md) to solve $r$ for the prime factors of $N$ using the post-processing from [[E21b]](https://doi.org/10.1007/s11128-021-03069-1):

In [8]:
from quaspy.orderfinding.general.postprocessing.ekera import solve_j_for_r;
from quaspy.factoring.general.postprocessing.ekera import solve_r_for_factors;

rp = solve_j_for_r(j, m, l, g, accept_multiple=True);

print("Solving j for a multiple r' of the order r yielded:", rp);

if (rp != None) and (rp != 1) and (r % rp == 0):
  print("\n[ OK ] A multiple r' of the order r was successfully recovered.");

  result = solve_r_for_factors(rp, N, timeout = 30);

  print("\nSolving r' for the complete factorization of N yielded:", result);

  if result == primes:
    print("\n[ OK ] All prime factors of N were successfully recovered.");
  else:
    print("\n[FAIL] Failed to completely factor N.");

else:
  print("\n[FAIL] Failed to recover a multiple r' of the order r.");

Solving j for a multiple r' of the order r yielded: 330134366551894315917706211506745509454137681362685148609860250547986162465011533419535711600812224766238714582604707704508079036389395396636051525846780252884470118538734150390482710149624849407859737276785042196048934244298016041119621484241144661856420585522512918167451442444076179265382182152421984220295189789777803421467401563188478968981153585261629719125535212182781767539055480364064755916943923987699238003430806209006608601217593396938152707716323503449779409075132509617875175160370679876988353097448890221593284142596996589083553155818504400542300154997549182805635804832311096818774420894736206567646929813054736978577529851146662118002060862796540542594945204783282355851000834284745392972066341482572655194748901953142885731110126821537827161420967331744077696061423188559655369194963192678426143395604086531385857940437494818811061099146126125174711280197906680237961346242141110373255414276185546080823680441572527552869211147347

## 3. Simulating order finding in $\mathbb Z_N^*$ heuristically to sample $g$ and $r$

In addition to the [<code>sample_r_given_N(N, factors)</code>](../docs/factoring/sampling/sample_r_given_N.md) function used above, [Quaspy](https://github.com/ekera/quaspy) also provides a function [<code>sample_g_r_given_N(N, N_factors, ..)</code>](../docs/factoring/sampling/sample_g_r_given_N.md) for sampling an element $g$ uniformly at random from $\mathbb Z_N^*$ and heuristically computing and returning its order $r$ alongside $g$.
For further details, see [[E21b]](https://doi.org/10.1007/s11128-021-03069-1) (see App. A).

Note that the [<code>sample_g_r_given_N(N, N_factors, ..)</code>](../docs/factoring/sampling/sample_g_r_given_N.md) function requires that the factorization of $N$ is known, as is the case here given that we selected the factors of $N$.
(If the function is also fed the factorization of $p_i - 1$ for $i \in [1, n] \cap \mathbb Z$ then the function can be made exact, as explained in [[E21b]](https://doi.org/10.1007/s11128-021-03069-1) and in the [documentation](../docs/factoring/sampling/sample_r_given_N.md).)

Below, we use said function to simulate the initial classical pre-processing step of Shor's algorithm where an element $g$ is selected uniformly at random from $\mathbb Z_N^*$.
This yields $g$ and $r$:

In [9]:
from quaspy.factoring.sampling import sample_g_r_given_N;

[g, r] = sample_g_r_given_N(N, factors);

print("Sampled g =", g, "\n");
print("Sampled r =", r, "\n");

Sampled g = 1420728730986955365321030102312337956677741618696152024966617432330422340932443920662157993144370660266900214300270485087238127425890066251165363810165718880951900109919412492350514524613000822325099019775753766771743093214412382392077227536591298853823595672865548949604750471762545904277340640656773809313682452228460365927112005788517279004411933292233332185003027992957568219741177109344483802147757366666248389277604056419293710374288759966223449722528802223433448253074790416557808120114319453181529563022744205403314973765557750197982755003094199636087553994499554466898641878574486268684872209902783685509677567475535579401853931219613763622141874388553961129450813017916711076617623727423877257822994324603402184994115775273901478079639418457248299108607176754332226003663897518826711159335250072526730069368941804019109041424333174038992801378335793625426654708638106365643068668119091340444876988434870876286247078887657561491594401840475274134182400179128781904848897337243028

### 3.1. Solving for the factors of $N$ in a single run
Let us first consider the setting where our goal is to solve for the factors of $N$ in a single run of the quantum part of Shor's order-finding algorithm.

#### 3.1.1. Sampling a frequency $j$
To start off, in analogy with Sect. 2, we use the [<code>sample_j_given_r(r, m, l, ..)</code>](../docs/orderfinding/general/sampling/sample_j_given_r.md) function provided by [Quaspy](https://github.com/ekera/quaspy) to sample the probability distribution induced by the quantum part of Shor's order-finding algorithm. This yields a frequency $j$ sampled from the probability distribution induced by the algorithm:

In [10]:
from quaspy.orderfinding.general.sampling import sample_j_given_r;

m = N.bit_length() - 1;
l = m; # Ensures that r^2 < 2^(m + l) as r < 2^m;

j = sample_j_given_r(r, m, l, timeout = 30);

print("Sampled j =", j);

Sampled j = 6028052116247312527270711343594726439495466956946997830066076462225203765390954171686024090420386152302624364896716575417723498935580137310477310914308789239146172598508789777298974867670152462097787646003359322229614068349075137293931028106571016336064723359448670750064455554964485566417345339607550830010537705706565742248492895246311023643140547847990662902923527255036201137599305487772425435591190567303919454796023980272055731920769332620989909609892029701174749479380974229001098556385868609290858171525282868178253524563166723541149073514202905403618709099052739256568542355040119575956527318459617308431460893894095366013009137750997486339680162571607466532120608385097821021138514237750987503462504159470537203596817312678676032612479003827536193602863593627759420964955395269431324372795001300953165444994390850925296634611285359959483208990530467115192535876654511462454484174346634039678295304548309577524700097253419707339112125735018452465337508640384941190983434314228719

#### 3.1.2. Solving the frequency $j$ and $g$ for the complete factorization of $N$
We may then use the [<code>solve_j_for_factors_mod_N(j, m, l, g, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_j_for_factors_mod_N.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to first solve $j$ and $g$ for a positive integer multiple of $r$ using the lattice-based post-processing from [[E24]](https://doi.org/10.1145/3655026), and to then solve $r$ for the prime factors of $N$ using the post-processing from [[E21b]](https://doi.org/10.1007/s11128-021-03069-1):

In [11]:
from quaspy.factoring.general.postprocessing.ekera import solve_j_for_factors_mod_N;

result = solve_j_for_factors_mod_N(j, m, l, g, N, timeout = 30);

print("Solving for all prime factors of N via [E21b] yielded:", result);

if result == primes:
  print("\n[ OK ] All prime factors of N were successfully recovered.");
else:
  print("\n[FAIL] Failed to completely factor the integer N.");

Solving for all prime factors of N via [E21b] yielded: {173612187966310048856533613919926723087410601579705334688318873344446043800050086085463822075235885182668014813003192310461613598155714480863906069098182045983142941158290019457107943528945346334700881758785143408420080471582745157047345541455016294240235708893368490542323437761887961141228197542063401693713, 98405689366658699793740774671388493146399314400366756950640922347844084251689300655531810820393726436218440023079778470944568991608249661166053747873038766882466781616445228750143893794510255140053414809393058202764441719028847452403794372876513937191571337099442917081948880823950633862187955099335442580059, 156913881488896793539425851866011077868395796772846710602560253453402314254064134697704979374584861997143601521530519128992332730503077660140706640878131189643875152158677158085019231148269289683451477255532537497954528933775907770617416686658637312536225636522693738347644676419707164328225753462105681945077, 144958118968

#### 3.1.3. Solving the frequency $j$ and $g$ for $r$
We may also use the [<code>solve_j_for_r_mod_N(j, m, l, g, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_j_for_r_mod_N.md) function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $j$ and $g$ for $r$ using the lattice-based post-processing from [[E24]](https://doi.org/10.1145/3655026).

In [12]:
from quaspy.orderfinding.general.postprocessing.ekera import solve_j_for_r_mod_N;

result = solve_j_for_r_mod_N(j, m, l, g, N, timeout = 30);

print("Solving j for r via [E22b] yielded:", result);

if result == r:
  print("\n[ OK ] The order r was successfully recovered.");
else:
  print("\n[FAIL] Failed to recover the order r.");

Solving j for r via [E22b] yielded: 3301343665518943159177062115067455094541376813626851486098602505479861624650115334195357116008122247662387145826047077045080790363893953966360515258467802528844701185387341503904827101496248494078597372767850421960489342442980160411196214842411446618564205855225129181674514424440761792653821821524219842202951897897778034214674015631884789689811535852616297191255352121827817675390554803640647559169439239876992380034308062090066086012175933969381527077163235034497794090751325096178751751603706798769883530974488902215932841425969965890835531558185044005423001549975491828056358048323110968187744208947362065676469298130547369785775298511466621180020608627965405425949452047832823558510008342847453929720663414825726551947489019531428857311101268215378271614209673317440776960614231885596553691949631926784261433956040865313858579404374948188110610991461261251747112801979066802379613462421411103732554142761855460808236804415725275528692111473478996186019394525

#### 3.1.4. Splitting $N$ given $g$ and $r$ using Shor's original algorithm
We may then split $N$ given $g$ and $r$ using Shor's original algorithm from [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700) for comparison.

To this end, we use the [<code>split_N_given_g_r(g, r, N)</code>](../docs/factoring/general/postprocessing/shor/split_N_given_g_r.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy).

In [13]:
from quaspy.factoring.general.postprocessing.shor import split_N_given_g_r;

result = split_N_given_g_r(g, r, N);

print("Attempting to split N using Shor's algorithm yielded:", result);

Attempting to split N using Shor's algorithm yielded: {173612187966310048856533613919926723087410601579705334688318873344446043800050086085463822075235885182668014813003192310461613598155714480863906069098182045983142941158290019457107943528945346334700881758785143408420080471582745157047345541455016294240235708893368490542323437761887961141228197542063401693713, 9084854679123189585739786608756663017077483602408069239174710779982949510234236760385256997837715018733961201289490868596372372235019792348931017422345463211944976484333628481054021403107616928989177476471638286423204782765828092219976823812505964610422075730363904300942737376860420980124898773308887700165958617597304942249995411303800424821684935905064229621846832304918697782671828998510141459214218212882783321234224122139777194679537795076474568422863879524455697049442878942580226773329723889517365610863899719085134331639330337839005909711983470005139970826706436901735038239879396837225648003826520596520172270788102602057071

### 3.2. Making tradeoffs and solving for the factors of $N$ in multiple runs
Let us now consider the setting where our goal is to solve for the factors of $N$ in multiple runs of the quantum part of Seifert's variation [[Seifert01]](https://doi.org/10.1007/3-540-45353-9_24) of Shor's order-finding algorithm [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700), as described in [[E24t]](https://diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sect. 5.4) and [[E21]](https://doi.org/10.1515/jmc-2020-0006) (see App. A), where the idea is to make tradeoffs by picking $\ell \approx m/s$ for $s$ some tradeoff factor instead of picking $\ell \sim m$.

As explained in [[E24t]](https://diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sect. 5.4) and [[E21]](https://doi.org/10.1515/jmc-2020-0006) (see App. A), each run of the quantum part of the algorithm yields at least $\sim \ell$ bits of information on the order $r$.
Hence, we expect to have to perform approximately $s$ runs to solve for $r$ efficiently and with high probability of success in the classical post-processing.

According to the estimates in [[E21]](https://doi.org/10.1515/jmc-2020-0006) (see Tabs. A1–A2) which were computed with the [Qunundrum](https://github.com/ekera/qunundrum) suite of MPI programs, when $m \ge 1024$, $s = 8$ and $\ell = \lceil m / s \rceil$, we need to make no more than $n = 9$ runs to solve efficiently in the classical post-processing with $\ge 99\%$ success probability without enumerating the lattice.
This when using the classical lattice-based post-processing from [[E21b]](https://doi.org/10.1007/s11128-021-03069-1), [[E24t]](https://diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sect. 5.4) and [[E21]](https://doi.org/10.1515/jmc-2020-0006) (see App. A) with supporting functions from [[E24]](https://doi.org/10.1145/3655026).
In the below example, we use this specific parameterization.

#### 3.2.1. Sampling $n$ frequencies $(j_1, \, \ldots, \, j_n)$
To start off, we proceed in analogy with Sect. 3.1.1 above to sample $n = 9$ frequencies $(j_1, \, \ldots, \, j_n)$ from the distribution induced by the quantum part of the algorithm.
To this end, we use the [<code>sample_j_given_r(r, m, l, ..)</code>](../docs/orderfinding/general/sampling/sample_j_given_r.md) function provided by [Quaspy](https://github.com/ekera/quaspy).

In [14]:
from quaspy.orderfinding.general.sampling import sample_j_given_r;

from math import ceil;

m = N.bit_length() - 1;
s = 8;
l = ceil(m / s);
n = 9;

j_list = [sample_j_given_r(r, m, l, timeout = 30) for _ in range(n)];

print("Sampled j =", j_list);

Sampled j = [...]


#### 3.2.2. Factoring $N$ completely given the $n$ frequencies $(j_1, \, \ldots, \, j_n)$ and $g$
We may then use the [<code>solve_multiple_j_for_factors_mod_N(j_list, m, l, g, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_multiple_j_for_factors_mod_N.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to factor $N$ completely given $(j_1, \, \ldots, \, j_n)$ and $g$.

This convenience function first calls the [<code>solve_multiple_j_for_r(j_list, m, l, g, ..)</code>](../docs/orderfinding/general/postprocessing/ekera/solve_multiple_j_for_r.md) function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $(j_1, \, \ldots, \, j_n)$ and $g$ for $r$, or for a positive integer multiple of $r$, using the lattice-based post-processing from [[E21b]](https://doi.org/10.1007/s11128-021-03069-1), [[E24t]](https://diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sect. 5.4) and [[E21]](https://doi.org/10.1515/jmc-2020-0006) (see App. A) with supporting functions from [[E24]](https://doi.org/10.1145/3655026). It then calls the [<code>solve_r_for_factors(r, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_r_for_factors.md) function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $r$ for the prime factors of $N$ using the post-processing from [[E21b]](https://doi.org/10.1007/s11128-021-03069-1).

In [ ]:
from quaspy.factoring.general.postprocessing.ekera import solve_multiple_j_for_factors_mod_N;

result = solve_multiple_j_for_factors_mod_N(j_list,
                                            m,
                                            l,
                                            g,
                                            N,
                                            enumerate = False,
                                            timeout = 600);

print("Solving for the prime factors yielded:", result);

if result == primes:
  print("\n[ OK ] All prime factors of N were successfully recovered.");
else:
  print("\n[FAIL] Failed to completely factor the integer N.");